# Lab 1 - 함수 정의 (Function Definition)로 Agent 만들기

이 노트북에서는 기능 정의에 대한 새로운 기능을 사용하여 Amazon Bedrock용 에이전트를 만들어 보겠습니다.

HR agent를 예로 들어 보겠습니다. 이 agent를 사용하면 사용 가능한 휴가 일수를 확인하고 새로운 휴가를 요청할 수 있습니다. 사용 가능한 휴가 일수를 확인하고 새로운 휴가를 확인하는 로직을 정의하기 위해 AWS Lambda 함수를 사용하겠습니다.

<img src="./images/lab1-architecture.png" alt="Create an Agent with Function Definition" style="height: 400px; width:950px;"/>

이 예제에서는 [SQLite](https://www.sqlite.org/) 데이터베이스에서 직원 데이터를 관리하고 에이전트 데모를 위한 합성 데이터를 생성하겠습니다.

## 전제 조건
시작하기 전에 최신 버전이 있는지 확인하기 위해 botocore 및 boto3 패키지를 업데이트해 보겠습니다.

In [37]:
!python3 -m pip install --upgrade -q botocore
!python3 -m pip install --upgrade -q boto3
!python3 -m pip install --upgrade -q awscli

이제 boto3 버전을 확인하여 올바른 버전이 설치되었는지 확인해 보겠습니다. 버전은 1.34.90 이상이어야 합니다.

In [46]:
import boto3
import json
import time
import zipfile
from io import BytesIO
import uuid
import pprint
import logging
print(boto3.__version__)

1.34.121


In [47]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [48]:
# getting boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

다음으로 에이전트와 생성 중인 람다 함수에 대한 몇 가지 구성 변수를 설정할 수 있습니다.

In [49]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '322537213286')

In [50]:
# configuration variables
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"
agent_description = "개인 휴가의 시간 관리를 위한 HR 에이전트"
agent_instruction = "너는 HR 에이전트로써, 직원들이 HR 정책을 이해하고, 개인 휴가 시간을 관리하는데 도움을 주도록 해"
agent_action_group_name = "VacationsActionGroup"
agent_action_group_description = "직원의 사용 가능한 개인 휴가 일수를 확인하고 새로운 휴가를 확인하기 위한 Actions"
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

## Lambda Function 만들기

SQLite 파일 `employee_database.db`와 연동하는 lambda function를 만들어 보겠습니다. 이를 위해 다음과 같이 하겠습니다:
1. 생성된 일부 데이터와 함께 직원 데이터베이스가 포함된 `employee_database.db` 파일을 만듭니다.
2. 람다 함수에 대한 로직이 포함된 `lambda_function.py` 파일을 만듭니다.
3. 람다 함수에 대한 IAM 역할을 생성합니다.
4. 필요한 권한으로 람다 함수 인프라를 생성합니다.

<img src="./images/HR_DB.png" alt="DB table schema" style="height: 400px; width:800px;"/>


In [52]:
# creating employee database to be used by lambda function
import sqlite3
import random
from datetime import date, timedelta

# Connect to the SQLite database (creates a new one if it doesn't exist)
conn = sqlite3.connect('employee_database.db')
c = conn.cursor()

# Create the employees table
c.execute('''CREATE TABLE IF NOT EXISTS employees
                (employee_id INTEGER PRIMARY KEY AUTOINCREMENT, employee_name TEXT, employee_job_title TEXT, employee_start_date TEXT, employee_employment_status TEXT)''')

# Create the vacations table
c.execute('''CREATE TABLE IF NOT EXISTS vacations
                (employee_id INTEGER, year INTEGER, employee_total_vacation_days INTEGER, employee_vacation_days_taken INTEGER, employee_vacation_days_available INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Create the planned_vacations table
c.execute('''CREATE TABLE IF NOT EXISTS planned_vacations
                (employee_id INTEGER, vacation_start_date TEXT, vacation_end_date TEXT, vacation_days_taken INTEGER, FOREIGN KEY(employee_id) REFERENCES employees(employee_id))''')

# Generate some random data for 10 employees
employee_names = ['John Doe', 'Jane Smith', 'Bob Johnson', 'Alice Williams', 'Tom Brown', 'Emily Davis', 'Michael Wilson', 'Sarah Taylor', 'David Anderson', 'Jessica Thompson']
job_titles = ['Manager', 'Developer', 'Designer', 'Analyst', 'Accountant', 'Sales Representative']
employment_statuses = ['Active', 'Inactive']

for i in range(10):
    name = employee_names[i]
    job_title = random.choice(job_titles)
    start_date = date(2015 + random.randint(0, 7), random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
    employment_status = random.choice(employment_statuses)
    c.execute("INSERT INTO employees (employee_name, employee_job_title, employee_start_date, employee_employment_status) VALUES (?, ?, ?, ?)", (name, job_title, start_date, employment_status))
    employee_id = c.lastrowid

    # Generate vacation data for the current employee
    for year in range(date.today().year, date.today().year - 3, -1):
        total_vacation_days = random.randint(10, 30)
        days_taken = random.randint(0, total_vacation_days)
        days_available = total_vacation_days - days_taken
        c.execute("INSERT INTO vacations (employee_id, year, employee_total_vacation_days, employee_vacation_days_taken, employee_vacation_days_available) VALUES (?, ?, ?, ?, ?)", (employee_id, year, total_vacation_days, days_taken, days_available))

        # Generate some planned vacations for the current employee and year
        num_planned_vacations = random.randint(0, 3)
        for _ in range(num_planned_vacations):
            start_date = date(year, random.randint(1, 12), random.randint(1, 28)).strftime('%Y-%m-%d')
            end_date = (date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])) + timedelta(days=random.randint(1, 14))).strftime('%Y-%m-%d')
            days_taken = (date(int(end_date[:4]), int(end_date[5:7]), int(end_date[8:])) - date(int(start_date[:4]), int(start_date[5:7]), int(start_date[8:])))
            c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, days_taken.days))

# Commit the changes and close the connection
conn.commit()
conn.close()

In [124]:
# def employee_db(employee_id):
#     import pandas as pd
#     import sqlite3

#     # Read sqlite query results into a pandas DataFrame
#     conn = sqlite3.connect('employee_database.db')
#     df = pd.read_sql_query(f"SELECT a.employee_id, \
#                            a.employee_name, \
#                            a.employee_job_title, \
#                            a.employee_start_date, \
#                            a.employee_employment_status, \
#                            b.year, \
#                            b.employee_total_vacation_days, \
#                            b.employee_vacation_days_taken, \
#                            b.employee_vacation_days_available, \
#                            c.vacation_start_date, \
#                            c.vacation_end_date, \
#                            c.vacation_days_taken \
#                            from employees a, vacations b, planned_vacations c \
#                            where a.employee_id=b.employee_id and b.employee_id=c.employee_id and a.employee_id={employee_id}", conn)
#     conn.close()
#     return df

이제 lambda 함수를 만들어 보겠습니다. 이 함수에는 주어진 employee_id에 대한 `get_available_vacations_days`의 기능과 시작일과 종료일을 제공하는 직원에 대한 `book_vacations`에 대한 기능을 구현합니다. 

두 작업 모두 이벤트를 통해 에이전트로부터 입력을 받는 AWS 람다 함수의 일부로 구현됩니다. 이벤트의 구조는 다음과 같습니다:

```html
{
    "messageVersion": "1.0", 
    "agent": {
        "alias": "<AGENT_ALIAS>", 
        "name": "hr-assistant-function-def", 
        "version": "<AGENT_VERSION>",
        "id": "<AGENT_ID>"
    }, 
    "sessionId": "<SESSION_ID>", 
    "sessionAttributes": {
        "<ATTRIBUTE_NAME>": "# Session attributes to be addressed in example 06-prompt-and-session-attributes"
    }, 
    "promptSessionAttributes": {
        "<PROMPT_NAME>": "# Session attributes to be addressed in example 06-prompt-and-session-attributes"
    }, 
    "inputText": "<USER_INPUT_TEXT>", 
    "actionGroup": "VacationsActionGroup", 
    "function": "<FUNCTION_TRIGGERED_BY_USER_INPUT_TEXT>", 
    "parameters": [{
        "<PARAM_1>": "<PARAM_1_VAL>", 
        "<PARAM_2>": "<PARAM_2_VAL>", 
        "<PARAM_N>": "<PARAM_N_VAL>"
    }]
}
```

In [53]:
%%writefile lambda_function.py
import os
import json
import shutil
import sqlite3
from datetime import datetime

def get_available_vacations_days(employee_id):
    # Connect to the SQLite database
    conn = sqlite3.connect('/tmp/employee_database.db')
    c = conn.cursor()

    if employee_id:

        # Fetch the available vacation days for the employee
        c.execute("""
            SELECT employee_vacation_days_available
            FROM vacations
            WHERE employee_id = ?
            ORDER BY year DESC
            LIMIT 1
        """, (employee_id,))

        available_vacation_days = c.fetchone()

        if available_vacation_days:
            available_vacation_days = available_vacation_days[0]  # Unpack the tuple
            print(f"Available vacation days for employed_id {employee_id}: {available_vacation_days}")
            conn.close()
            return available_vacation_days
        else:
            return_msg = f"No vacation data found for employed_id {employee_id}"
            print(return_msg)
            return return_msg
            conn.close()
    else:
        raise Exception(f"No employeed id provided")

    # Close the database connection
    conn.close()
    
    
def reserve_vacation_time(employee_id, start_date, end_date):
    # Connect to the SQLite database

    conn = sqlite3.connect('/tmp/employee_database.db')
    c = conn.cursor()
    try:
        # Calculate the number of vacation days
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        vacation_days = (end_date - start_date).days + 1

        # Get the current year
        current_year = start_date.year

        # Check if the employee exists
        c.execute("SELECT * FROM employees WHERE employee_id = ?", (employee_id,))
        employee = c.fetchone()
        if employee is None:
            return_msg = f"Employee with ID {employee_id} does not exist."
            print(return_msg)
            conn.close()
            return return_msg

        # Check if the vacation days are available for the employee in the current year
        c.execute("SELECT employee_vacation_days_available FROM vacations WHERE employee_id = ? AND year = ?", (employee_id, current_year))
        available_days = c.fetchone()
        if available_days is None or available_days[0] < vacation_days:
            return_msg = f"Employee with ID {employee_id} does not have enough vacation days available for the requested period."
            print(return_msg)
            conn.close()
            return return_msg

        # Insert the new vacation into the planned_vacations table
        c.execute("INSERT INTO planned_vacations (employee_id, vacation_start_date, vacation_end_date, vacation_days_taken) VALUES (?, ?, ?, ?)", (employee_id, start_date, end_date, vacation_days))

        # Update the vacations table with the new vacation days taken
        c.execute("UPDATE vacations SET employee_vacation_days_taken = employee_vacation_days_taken + ?, employee_vacation_days_available = employee_vacation_days_available - ? WHERE employee_id = ? AND year = ?", (vacation_days, vacation_days, employee_id, current_year))

        conn.commit()
        print(f"Vacation saved successfully for employee with ID {employee_id} from {start_date} to {end_date}.")
        # Close the database connection
        conn.close()
        return f"Vacation saved successfully for employee with ID {employee_id} from {start_date} to {end_date}."
    except Exception as e:
        raise Exception(f"Error occurred: {e}")
        conn.rollback()
        # Close the database connection
        conn.close()
        return f"Error occurred: {e}"
        

def lambda_handler(event, context):
    original_db_file = 'employee_database.db'
    target_db_file = '/tmp/employee_database.db'
    if not os.path.exists(target_db_file):
        shutil.copy2(original_db_file, target_db_file)
    
    agent = event['agent']
    actionGroup = event['actionGroup']
    function = event['function']
    parameters = event.get('parameters', [])
    responseBody =  {
        "TEXT": {
            "body": "Error, no function was called"
        }
    }


    
    if function == 'get_available_vacations_days':
        employee_id = None
        for param in parameters:
            if param["name"] == "employee_id":
                employee_id = param["value"]

        if not employee_id:
            raise Exception("Missing mandatory parameter: employee_id")
        vacation_days = get_available_vacations_days(employee_id)
        responseBody =  {
            'TEXT': {
                "body": f"available vacation days for employed_id {employee_id}: {vacation_days}"
            }
        }
    elif function == 'reserve_vacation_time':
        employee_id = None
        start_date = None
        end_date = None
        for param in parameters:
            if param["name"] == "employee_id":
                employee_id = param["value"]
            if param["name"] == "start_date":
                start_date = param["value"]
            if param["name"] == "end_date":
                end_date = param["value"]
            
        if not employee_id:
            raise Exception("Missing mandatory parameter: employee_id")
        if not start_date:
            raise Exception("Missing mandatory parameter: start_date")
        if not end_date:
            raise Exception("Missing mandatory parameter: end_date")
        
        completion_message = reserve_vacation_time(employee_id, start_date, end_date)
        responseBody =  {
            'TEXT': {
                "body": completion_message
            }
        }  
    action_response = {
        'actionGroup': actionGroup,
        'function': function,
        'functionResponse': {
            'responseBody': responseBody
        }

    }

    function_response = {'response': action_response, 'messageVersion': event['messageVersion']}
    print("Response: {}".format(function_response))

    return function_response


Overwriting lambda_function.py


다음으로 Bedrock 모델을 호출하기 위한 lambda IAM 역할 및 정책을 만들어 보겠습니다.

In [54]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_function_role,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_function_role)

iam_client.attach_role_policy(
    RoleName=lambda_function_role,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': '962a63c5-ac6e-45d2-a93a-fb9696599fb7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 05:51:50 GMT',
   'x-amzn-requestid': '962a63c5-ac6e-45d2-a93a-fb9696599fb7',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

이제 람다 함수를 Zip 파일로 패키징하고 boto3를 사용하여 람다 함수를 만들 수 있습니다.

In [55]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write("lambda_function.py")
z.write("employee_database.db")
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_function_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

## Agent 만들기
이제 agent를 만들겠습니다. 그러기 위해서는 먼저 특정 foundation 모델에 대한 Bedrock 모델 호출을 허용하는 agent 정책과 이에 연결된 정책으로 agent IAM 역할을 만들어야 합니다. 

In [56]:
# Create IAM policies for agent
bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



In [57]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '01d17796-adb5-449d-b37d-c8ea77cd37f9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 05:53:32 GMT',
   'x-amzn-requestid': '01d17796-adb5-449d-b37d-c8ea77cd37f9',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

### Agent 만들기
필요한 IAM 역할이 만들어지면 Bedrock agent 클라이언트를 사용하여 새 agent를 만들 수 있습니다. 이를 위해 `create_agent` 함수를 사용합니다. 여기에는 agent 이름, 기본 foundation 모델 및 instructions이 필요합니다. agent 설명을 제공할 수도 있습니다. 생성된 agent는 아직 준비되지 않은 상태입니다. 나중에 agent를 준비하여 사용할 것입니다.

In [58]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': '91b673dd-4e33-4ccd-be42-e1b9601130da',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 05:55:05 GMT',
   'content-type': 'application/json',
   'content-length': '690',
   'connection': 'keep-alive',
   'x-amzn-requestid': '91b673dd-4e33-4ccd-be42-e1b9601130da',
   'x-amz-apigw-id': 'Y-4M8EtPPHcEhUA=',
   'x-amzn-trace-id': 'Root=1-6662a0b8-6e7fc2cf5242e75c01007fbd'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:322537213286:agent/SOLQFHSMWN',
  'agentId': 'SOLQFHSMWN',
  'agentName': 'hr-assistant-function-def',
  'agentResourceRoleArn': 'arn:aws:iam::322537213286:role/AmazonBedrockExecutionRoleForAgents_hr-assistant-function-def',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 6, 7, 5, 55, 5, 54492, tzinfo=tzlocal()),
  'description': '개인 휴가의 시간 관리를 위한 HR 에이전트',
  'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'idleSessionTTLInSeconds': 1800,
  'instruction': '너는

이제 agent id를 로컬 변수에 저장하여 후속 단계에서 사용하도록 하겠습니다.

In [59]:
agent_id = response['agent']['agentId']
agent_id

'SOLQFHSMWN'

## Agent Action Group 만들기
이제 앞서 만든 lambda 함수를 사용하는 agent action group을 만들어 보겠습니다. 이 기능은 [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) 함수가 제공합니다. 아직 agent version이나 alias을 만들지 않았으므로 `DRAFT`를 agent version으로 사용하겠습니다. agent에게 action group 기능에 대해 알리기 위해 action group 설명을 제공합니다.

이 예에서는 `functionSchema`를 사용하여 action group 기능을 제공합니다. 또는 `APISchema`를 제공할 수도 있습니다. 노트북 [02-create-agent-with-api-schema.ipynb](02-create-agent-with-api-schema/02-create-agent-with-api-schema.ipynb)는 이러한 접근 방식의 예시를 제공합니다.

함수 스키마를 사용하여 함수를 정의하려면 각 함수에 대한 `name`, `description`와 `parameters`를 제공해야 합니다.

In [60]:
agent_functions = [
    {
        'name': 'get_available_vacations_days',
        'description': 'get the number of vacations available for a certain employee',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee to get the available vacations",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'reserve_vacation_time',
        'description': 'reserve vacation time for a specific employee',
        'parameters': {
            "employee_id": {
                "description": "the id of the employee for which time off will be reserved",
                "required": True,
                "type": "integer"
            },
            "start_date": {
                "description": "the start date for the vacation time",
                "required": True,
                "type": "string"
            },
            "end_date": {
                "description": "the end date for the vacation time",
                "required": True,
                "type": "string"
            }
        }
    },
]

In [61]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [62]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': 'd3bebd87-d01e-4991-9c3a-398e74ee3108',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 06:16:54 GMT',
   'content-type': 'application/json',
   'content-length': '1230',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd3bebd87-d01e-4991-9c3a-398e74ee3108',
   'x-amz-apigw-id': 'Y-7ZfEuWPHcEgew=',
   'x-amzn-trace-id': 'Root=1-6662a5d6-69a6dedc34306df04e84e54d'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:322537213286:function:hr-assistant-function-def-us-west-2-322537213286'},
  'actionGroupId': 'PY5PT9AGHV',
  'actionGroupName': 'VacationsActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'SOLQFHSMWN',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 6, 7, 6, 16, 54, 244130, tzinfo=tzlocal()),
  'description': '직원의 사용 가능한 개인 휴가 일수를 확인하고 새로운 휴가를 확인하기 위한 Actions',
  'functionSchema': {'functions': [{'description': 'get the number of vaca

## Agent이 Action Group Lambda를 호출하도록 허용하기
action group을 사용하기 전에 agent이 action group과 연결된 람다 함수를 호출할 수 있도록 허용해야 합니다. 이는 리소스 기반 정책을 통해 이루어집니다. 생성한 람다 함수에 리소스 기반 정책을 추가해 보겠습니다.

In [63]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


In [64]:
response

{'ResponseMetadata': {'RequestId': '3c97f5d8-7839-45c1-968f-e0a03c9764a1',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 06:41:31 GMT',
   'content-type': 'application/json',
   'content-length': '376',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3c97f5d8-7839-45c1-968f-e0a03c9764a1'},
  'RetryAttempts': 1},
 'Statement': '{"Sid":"allow_bedrock","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-west-2:322537213286:function:hr-assistant-function-def-us-west-2-322537213286","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-west-2:322537213286:agent/SOLQFHSMWN"}}}'}

## Agent 준비하기

내부 테스트에 사용할 수 있는 agent의 DRAFT 버전을 만들어 보겠습니다.


In [65]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)

{'ResponseMetadata': {'RequestId': '4c0812a5-43e6-4232-a9ed-91237da62137', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 06:42:10 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '4c0812a5-43e6-4232-a9ed-91237da62137', 'x-amz-apigw-id': 'Y-_GaH79vHcESSg=', 'x-amzn-trace-id': 'Root=1-6662abc2-57c8c1a66af9f79d0f8398d6'}, 'RetryAttempts': 0}, 'agentId': 'SOLQFHSMWN', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 6, 7, 6, 42, 10, 386796, tzinfo=tzlocal())}


In [66]:
# Pause to make sure agent is prepared
time.sleep(30)

# Extract the agentAliasId from the response
agent_alias_id = "TSTALIASID"

## Agent 호출

이제 agent를 만들었으니 `bedrock-agent-runtime` 클라이언트를 사용하여 이 agent를 호출하고 몇 가지 작업을 수행해 보겠습니다.

In [92]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    # inputText="How much vacation does employee_id 1 have available?",
    inputText="직원 1은 얼마나 많은 휴가를 사용할 수 있나요?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-06-07 07:20:23,231] p30181 {2145766712.py:17} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 07 Jun 2024 07:20:23 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '677dbbec-249e-11ef-a3f9-0a86a609e469',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'b24f14ab-3621-4dbf-a1ef-1fbf9159c2d1'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'b24f14ab-3621-4dbf-a1ef-1fbf9159c2d1',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fd46e9378b0>,
 'contentType': 'application/json',
 'sessionId': '677dbbec-249e-11ef-a3f9-0a86a609e469'}


In [93]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-06-07 07:20:35,217] p30181 {<timed exec>:6} INFO - Final answer ->
직원 1은 16일의 휴가를 사용할 수 있습니다.


CPU times: user 0 ns, sys: 2.77 ms, total: 2.77 ms
Wall time: 10.5 s


In [94]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

직원 1은 16일의 휴가를 사용할 수 있습니다.


In [95]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    # inputText="great. please reserve one day of time off, June 1 2024",
    inputText="그럼, 2024년 6월 1일 하루 휴가를 예약해 주세요.",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-06-07 07:20:48,509] p30181 {4024832624.py:11} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 07 Jun 2024 07:20:48 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '677dbbec-249e-11ef-a3f9-0a86a609e469',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'c339cd67-4dd3-4c97-8978-31a96506bda5'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'c339cd67-4dd3-4c97-8978-31a96506bda5',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fd46e937be0>,
 'contentType': 'application/json',
 'sessionId': '677dbbec-249e-11ef-a3f9-0a86a609e469'}


In [96]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-06-07 07:20:56,583] p30181 {<timed exec>:6} INFO - Final answer ->
직원 1의 2024년 6월 1일 하루 휴가가 예약되었습니다.


CPU times: user 1.22 ms, sys: 1.31 ms, total: 2.53 ms
Wall time: 6.85 s


In [97]:
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    # inputText="now let me take the last 3 months of the year off as vacation, from Oct 1 2024 through Dec 31 2024",
    inputText="2024년 10월 1일부터 2024년 12월 31일까지 1년 중 마지막 3개월을 휴가로 사용하고 싶어요",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-06-07 07:20:56,628] p30181 {2577472011.py:11} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 07 Jun 2024 07:20:56 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '677dbbec-249e-11ef-a3f9-0a86a609e469',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '3cf5ac50-c56f-4248-a2ce-30d618d9607d'},
                      'HTTPStatusCode': 200,
                      'RequestId': '3cf5ac50-c56f-4248-a2ce-30d618d9607d',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fd46e936d70>,
 'contentType': 'application/json',
 'sessionId': '677dbbec-249e-11ef-a3f9-0a86a609e469'}


In [98]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-06-07 07:21:13,968] p30181 {<timed exec>:6} INFO - Final answer ->
죄송합니다. 직원 1은 2024년 10월 1일부터 2024년 12월 31일까지 3개월 휴가를 사용할 수 있는 충분한 휴가 일수가 없습니다. 현재 남은 휴가 일수는 15일입니다.


CPU times: user 2.19 ms, sys: 360 µs, total: 2.55 ms
Wall time: 17.3 s


In [99]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    logger.info(pprint.pprint(agentResponse))
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [100]:
simple_agent_invoke("직원 2의 휴가 기간은 얼마나 되나요?", agent_id, agent_alias_id, session_id)

[2024-06-07 07:21:14,022] p30181 {1728146489.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 07 Jun 2024 07:21:14 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '677dbbec-249e-11ef-a3f9-0a86a609e469',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'b7f7dcbd-1cf3-4d41-b5dd-3e5aa2c5dd9c'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'b7f7dcbd-1cf3-4d41-b5dd-3e5aa2c5dd9c',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fd46e9355d0>,
 'contentType': 'application/json',
 'sessionId': '677dbbec-249e-11ef-a3f9-0a86a609e469'}


[2024-06-07 07:21:24,884] p30181 {1728146489.py:17} INFO - Final answer ->
직원 2는 11일의 휴가를 사용할 수 있습니다.


In [101]:
simple_agent_invoke("2024년 7월 30일부터 2024년 8월 4일까지 예약해 주세요.", agent_id, agent_alias_id, session_id)

[2024-06-07 07:21:37,715] p30181 {1728146489.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 07 Jun 2024 07:21:37 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '677dbbec-249e-11ef-a3f9-0a86a609e469',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '60d447bd-6f1c-40a1-bdb0-24589011914d'},
                      'HTTPStatusCode': 200,
                      'RequestId': '60d447bd-6f1c-40a1-bdb0-24589011914d',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fd46e937b20>,
 'contentType': 'application/json',
 'sessionId': '677dbbec-249e-11ef-a3f9-0a86a609e469'}


[2024-06-07 07:21:45,434] p30181 {1728146489.py:17} INFO - Final answer ->
직원 2의 2024년 7월 30일부터 2024년 8월 4일까지의 휴가가 예약되었습니다.


In [102]:
simple_agent_invoke("직원 9는 휴가가 얼마나 남았나요?", agent_id, agent_alias_id, session_id, enable_trace=True)

[2024-06-07 07:22:28,899] p30181 {1728146489.py:10} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 07 Jun 2024 07:22:28 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '677dbbec-249e-11ef-a3f9-0a86a609e469',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '58cabd3d-52a3-443f-bc58-db85316b3310'},
                      'HTTPStatusCode': 200,
                      'RequestId': '58cabd3d-52a3-443f-bc58-db85316b3310',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7fd46e935a20>,
 'contentType': 'application/json',
 'sessionId': '677dbbec-249e-11ef-a3f9-0a86a609e469'}


[2024-06-07 07:22:29,151] p30181 {1728146489.py:22} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "SOLQFHSMWN",
  "agentVersion": "DRAFT",
  "sessionId": "677dbbec-249e-11ef-a3f9-0a86a609e469",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [
            "</invoke>",
            "</answer>",
            "</error>"
          ],
          "temperature": 0.0,
          "topK": 250,
          "topP": 1.0
        },
        "text": "{\"system\":\" \ub108\ub294 HR \uc5d0\uc774\uc804\ud2b8\ub85c\uc368, \uc9c1\uc6d0\ub4e4\uc774 HR \uc815\ucc45\uc744 \uc774\ud574\ud558\uace0, \uac1c\uc778 \ud734\uac00 \uc2dc\uac04\uc744 \uad00\ub9ac\ud558\ub294\ub370 \ub3c4\uc6c0\uc744 \uc8fc\ub3c4\ub85d \ud574 You have been provided with a set of functions to answer the user's question. You must call the functions in the format below: <function_calls> <invoke> <tool_name>$TOOL_NAME<

## Clean up (optional)

The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:

1. update the action group to disable it
2. delete agent action group
4. delete agent
5. delete lambda function
6. delete the created IAM roles and policies


In [120]:
# This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first
action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    functionSchema={
        'functions': agent_functions
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [121]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [122]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=lambda_function_name
)

{'ResponseMetadata': {'RequestId': '08cd21d9-ae9d-44c7-8399-8474a07ff865',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 07:40:11 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': '08cd21d9-ae9d-44c7-8399-8474a07ff865'},
  'RetryAttempts': 1}}

In [123]:
# Delete IAM Roles and policies

for policy in [agent_bedrock_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_function_role, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_function_role]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)


## Conclusion
We have now experimented with using boto3 SDK to create, invoke and delete an agent created using function definitions.

## Take aways
Adapt this notebook to create new agents using function definitions for your application

## Thank You